# Benchmark of various outlier detection model thresholders

### The models are evaluated by ROC, Precision @ n and execution time on 17 benchmark datasets. All datasets are split (60% for training and 40% for testing). The full result by averaging 10 indepent trials can be found [here]

The thresholders covered in this example include:

1. **IQR: Inter-Quartile Region** 
2. **MAD: Median Absolute Deviation**
3. **FWFM: Full Width at Full Minimum**
4. **YJ: Yeo-Johnson Transformation**
5. **ZSCORE: Z Score**
6. **AUCP: Area Under the Curve Percentage**
7. **QMCD: Quasi-Monte Carlo Discreperancy**
8. **FGD: Fixed Gradient Descent**
9. **DSN: Distance Shift from Normal**
10. **CLF: Trained Classifier**
11. **FILTER: Filtering Based**
12. **WIND: Topological Winding Number**
13. **EB: Elliptical Boundary**
14. **REGR: Regression Intercept**
15. **BOOT: Bootstrap Method**
16. **MCST: Monte Carlo Statistical Tests**
17. **HIST: Histogram Based Methods**
18. **MOLL: Mollifier**
19. **CHAU: Chauvenet's Criterion**
20. **GESD: Generalized Extreme Studentized Deviate**
21. **MTT: Modified Thompson Tau Test**
22. **KARCH: Karcher Mean**
23. **OCSVM: One-Class SVM**
24. **CLUST: Clustering**
25. **DECOMP: Decomposition**
26. **META: Meta-model**
27. **VAE: Variational Autoencoder**
28. **CPD: Change Point Detection**

In [1]:
from __future__ import division
from __future__ import print_function

import os
import sys
from time import time

# temporary solution for relative imports in case pyod is not installed
# if pyod is installed, no need to use the following line
sys.path.append(
    os.path.abspath(os.path.join(os.path.dirname("__file__"), '..')))

import numpy as np
from numpy import percentile
import matplotlib.pyplot as plt
import matplotlib.font_manager
import pandas as pd
from sklearn.model_selection import train_test_split
from scipy.io import loadmat

from pyod.models.sampling import Sampling
from pyod.models.kde import KDE
from pyod.models.knn import KNN
from pyod.models.mcd import MCD
from pyod.models.iforest import IForest

from pythresh.thresholds.iqr import IQR
from pythresh.thresholds.mad import MAD
from pythresh.thresholds.fwfm import FWFM
from pythresh.thresholds.yj import YJ
from pythresh.thresholds.zscore import ZSCORE
from pythresh.thresholds.aucp import AUCP
from pythresh.thresholds.qmcd import QMCD
from pythresh.thresholds.fgd import FGD
from pythresh.thresholds.dsn import DSN
from pythresh.thresholds.clf import CLF
from pythresh.thresholds.filter import FILTER
from pythresh.thresholds.wind import WIND
from pythresh.thresholds.eb import EB
from pythresh.thresholds.regr import REGR
from pythresh.thresholds.boot import BOOT
from pythresh.thresholds.mcst import MCST
from pythresh.thresholds.hist import HIST
from pythresh.thresholds.moll import MOLL
from pythresh.thresholds.chau import CHAU
from pythresh.thresholds.gesd import GESD
from pythresh.thresholds.mtt import MTT
from pythresh.thresholds.karch import KARCH
from pythresh.thresholds.ocsvm import OCSVM
from pythresh.thresholds.clust import CLUST
from pythresh.thresholds.decomp import DECOMP
from pythresh.thresholds.meta import META
from pythresh.thresholds.vae import VAE
from pythresh.thresholds.cpd import CPD

from pyod.utils.utility import standardizer
from pyod.utils.utility import precision_n_scores
from sklearn.metrics import roc_auc_score

INFO: Using numpy backend


In [2]:
# Define data file and read X and y
mat_file_list = ['arrhythmia.mat',
                 'cardio.mat',
                 'glass.mat',
                 'ionosphere.mat',
                 'letter.mat',
                 'lympho.mat',
                 'mnist.mat',
                 'musk.mat',
                 'optdigits.mat',
                 'pendigits.mat',
                 'pima.mat',
                 'satellite.mat',
                 'satimage-2.mat',
                 'vertebral.mat',
                 'vowels.mat',
                 'wbc.mat']

# Define nine outlier detection tools to be compared
random_state = np.random.RandomState(42)

df_columns = ['Data', '#Samples', '# Dimensions', 'Outlier Perc', 'IQR', 'MAD', 'FWFM', 
              'YJ', 'ZSCORE', 'AUCP', 'QMCD', 'FGD', 'DSN', 'CLF', 'FILTER', 'WIND', 
              'EB', 'REGR', 'BOOT', 'MCST', 'HIST', 'MOLL', 'CHAU', 'GESD', 'MTT', 'KARCH',
              'OCSVM', 'CLUST', 'DECOMP', 'META', 'VAE', 'CPD']
roc_df = pd.DataFrame(columns=df_columns)
prn_df = pd.DataFrame(columns=df_columns)
time_df = pd.DataFrame(columns=df_columns)

clf = IForest()


for mat_file in mat_file_list:
    print("\n... Processing", mat_file, '...')
    mat = loadmat(os.path.join('data', mat_file))

    X = mat['X']
    y = mat['y'].ravel()
    outliers_fraction = np.count_nonzero(y) / len(y)
    outliers_percentage = round(outliers_fraction * 100, ndigits=4)

    # construct containers for saving results
    roc_list = [mat_file[:-4], X.shape[0], X.shape[1], outliers_percentage]
    prn_list = [mat_file[:-4], X.shape[0], X.shape[1], outliers_percentage]
    time_list = [mat_file[:-4], X.shape[0], X.shape[1], outliers_percentage]

    # 60% data for training and 40% for testing
    X_train, y_train, X_test, y_test = train_test_split(X, y, test_size=0.4,
                                                        random_state=random_state)

    # standardizing data for processing
    X_train_norm, X_test_norm = standardizer(X_train, X_test)

    thresholders = {
        'Inter-Quartile Region (IQR)':IQR(),
        'Median Absolute Deviation (MAD)':MAD(),
        'Full Width at Full Minimum (FWFM)':FWFM(),
        'Yeo-Johnson Transformation (YJ)': YJ(),
        'Z Score (ZSCORE)': ZSCORE(),
        'AUC Percentage (AUCP)': AUCP(),
        'Quasi-Monte Carlo Discreperancy (QMCD)': QMCD(),
        'Fixed Gradient Descent (FGD)': FGD(),
        'Distance Shift from Normal (DSN)': DSN(),
        'Trained Classifier (CLF)': CLF(),
        'Filtering Based (FILTER)': FILTER(),
        'Topological Winding Number (WIND)': WIND(),
        'Elliptical Boundary (EB)': EB(),
        'Regression Intercept (REGR)': REGR(),
        'Bootstrap Method (BOOT)': BOOT(),
        'Monte Carlo Statistical Tests (MCST)': MCST(),
        'Histogram Based Methods (HIST)': HIST(),
        'Mollifier (MOLL)': MOLL(),
        "Chauvenet's Criterion (CHAU)": CHAU(),
        'Generalized Extreme Studentized Deviate (GESD)': GESD(),
        'Modified Thompson Tau Test (MTT)': MTT(),
        'Karcher Mean (KARCH)': KARCH(),
        'One-Class SVM (OCSVM)': OCSVM(),
        'Clustering (CLUST)': CLUST(),
        'Decomposition (DECOMP)': DECOMP(),
        'Meta-model (META)': META(),
        'Variational Autoencoder (VAE)': VAE(),
        'Change Point Detection (CPD)': CPD()
    }
    
    clf.fit(X_train_norm)
    scores = clf.decision_scores_
    
    for thres_name, thres in thresholders.items():
        t0 = time()
        pred = thres.eval(scores)
        contam = np.sum(pred)/len(pred)
        
        if contam<=0: contam=1e-3
        if contam>0.5: contam=0.5
        
        clf = IForest(contamination=contam)
        clf.fit(X_train_norm)
        test_scores = clf.decision_function(X_test_norm)
        t1 = time()
        duration = round(t1 - t0, ndigits=4)
        time_list.append(duration)

        roc = round(roc_auc_score(y_test, test_scores), ndigits=4)
        prn = round(precision_n_scores(y_test, test_scores), ndigits=4)

        roc_list.append(roc)
        prn_list.append(prn)

    temp_df = pd.DataFrame(time_list).transpose()
    temp_df.columns = df_columns
    time_df = pd.concat([time_df, temp_df], axis=0)

    temp_df = pd.DataFrame(roc_list).transpose()
    temp_df.columns = df_columns
    roc_df = pd.concat([roc_df, temp_df], axis=0)

    temp_df = pd.DataFrame(prn_list).transpose()
    temp_df.columns = df_columns
    prn_df = pd.concat([prn_df, temp_df], axis=0)


... Processing arrhythmia.mat ...

... Processing cardio.mat ...

... Processing glass.mat ...

... Processing ionosphere.mat ...

... Processing letter.mat ...

... Processing lympho.mat ...

... Processing mnist.mat ...

... Processing musk.mat ...

... Processing optdigits.mat ...

... Processing pendigits.mat ...

... Processing pima.mat ...

... Processing satellite.mat ...

... Processing satimage-2.mat ...

... Processing vertebral.mat ...

... Processing vowels.mat ...

... Processing wbc.mat ...


In [5]:
print('Time complexity')
time_df

Time complexity


,Data,#Samples,# Dimensions,Outlier Perc,IQR,MAD,FWFM,YJ,ZSCORE,AUCP,...,CHAU,GESD,MTT,KARCH,OCSVM,CLUST,DECOMP,META,VAE,CPD
0,arrhythmia,452,274,14.6018,0.7082,0.8813,0.7858,0.9174,0.7127,0.7662,...,0.6553,0.7276,0.6663,0.6599,0.6579,0.8977,0.7949,8.5686,3.4201,0.7566
0,cardio,1831,21,9.6122,0.6503,0.7372,0.7563,1.476,0.6433,0.8447,...,0.6449,0.9077,0.6695,0.7051,0.6476,0.9969,0.9603,5.8997,11.0554,1.0886
0,glass,214,9,4.2056,0.4707,0.5031,0.5159,0.7308,0.4644,0.481,...,0.4646,0.4928,0.4673,0.4662,0.4684,0.6025,0.4839,5.4491,1.6761,0.5023
0,ionosphere,351,33,35.8974,0.544,0.581,0.513,0.8085,0.5008,0.5312,...,0.5004,0.5486,0.5019,0.5043,0.5126,0.6045,0.5365,5.5592,2.6322,0.5582
0,letter,1600,32,6.25,0.6638,0.7384,0.7378,1.2446,0.6571,0.8511,...,0.651,0.8731,0.6592,0.6867,0.6544,0.8898,0.9129,5.8478,9.5795,1.025
0,lympho,148,18,4.0541,0.4647,0.487,0.5089,0.7683,0.4715,0.4707,...,0.4584,0.4801,0.4596,0.462,0.4625,0.4807,0.4837,5.3553,1.4143,0.4818
0,mnist,7603,100,9.2069,1.9991,1.9978,4.1311,5.2015,1.9942,4.0043,...,1.9961,3.1136,2.047,2.7899,2.0192,8.9813,5.5401,8.6803,46.744,6.2205
0,musk,3062,166,3.1679,1.4493,1.3912,1.7022,2.4241,1.4182,1.8461,...,1.3953,1.8954,1.4046,1.5268,1.3947,2.1786,2.1313,7.0016,19.1604,2.3325
0,optdigits,5216,64,2.8758,1.3666,1.3344,2.3899,2.9812,1.3323,2.4844,...,1.3351,2.0773,1.3636,1.7099,1.3374,3.9888,3.1051,7.349,31.8961,3.5334
0,pendigits,6870,16,2.2707,1.1708,1.1697,2.767,3.815,1.1046,2.7553,...,1.1004,2.0933,1.1389,1.7481,1.1087,6.384,4.1818,7.6502,41.144,4.767


In [6]:
print('ROC Performance')
roc_df

ROC Performance


,Data,#Samples,# Dimensions,Outlier Perc,IQR,MAD,FWFM,YJ,ZSCORE,AUCP,...,CHAU,GESD,MTT,KARCH,OCSVM,CLUST,DECOMP,META,VAE,CPD
0,arrhythmia,452,274,14.6018,0.8224,0.8277,0.8205,0.8518,0.8751,0.8394,...,0.8415,0.848,0.8406,0.8532,0.8471,0.8084,0.8408,0.8303,0.8217,0.8592
0,cardio,1831,21,9.6122,0.9072,0.9273,0.9307,0.9272,0.9183,0.9036,...,0.906,0.9476,0.9261,0.926,0.9237,0.9199,0.9354,0.9243,0.9254,0.9009
0,glass,214,9,4.2056,0.6321,0.6222,0.6691,0.6247,0.6222,0.6,...,0.6222,0.6741,0.6519,0.6667,0.6296,0.6444,0.6444,0.6617,0.6444,0.6395
0,ionosphere,351,33,35.8974,0.7796,0.784,0.7876,0.7764,0.7725,0.7941,...,0.7844,0.792,0.7883,0.7941,0.7854,0.773,0.7748,0.7767,0.7737,0.7886
0,letter,1600,32,6.25,0.6138,0.6186,0.6067,0.6488,0.6574,0.6065,...,0.5978,0.646,0.6142,0.6368,0.6382,0.5984,0.5965,0.5987,0.6038,0.6591
0,lympho,148,18,4.0541,0.9942,1.0,1.0,1.0,0.9942,1.0,...,0.9942,1.0,1.0,1.0,0.9942,1.0,1.0,1.0,1.0,1.0
0,mnist,7603,100,9.2069,0.8267,0.7958,0.7797,0.8035,0.7926,0.8116,...,0.8283,0.7975,0.7617,0.8379,0.8015,0.7921,0.7577,0.7582,0.7968,0.793
0,musk,3062,166,3.1679,0.9972,0.9998,0.9947,0.9996,1.0,0.9951,...,0.9997,1.0,0.9996,1.0,1.0,1.0,1.0,0.9998,0.9992,1.0
0,optdigits,5216,64,2.8758,0.7226,0.7979,0.7297,0.7601,0.6249,0.7357,...,0.7408,0.7685,0.7519,0.7255,0.681,0.6864,0.7213,0.7003,0.729,0.7987
0,pendigits,6870,16,2.2707,0.9417,0.9498,0.9419,0.9362,0.9211,0.9583,...,0.9558,0.9299,0.947,0.928,0.941,0.9397,0.9344,0.9533,0.9393,0.9345


In [7]:
print('Precision @ n Performance')
prn_df

Precision @ n Performance


,Data,#Samples,# Dimensions,Outlier Perc,IQR,MAD,FWFM,YJ,ZSCORE,AUCP,...,CHAU,GESD,MTT,KARCH,OCSVM,CLUST,DECOMP,META,VAE,CPD
0,arrhythmia,452,274,14.6018,0.5357,0.6071,0.5357,0.6071,0.6071,0.5357,...,0.5714,0.5714,0.5,0.5,0.5714,0.5,0.5714,0.5357,0.5357,0.5714
0,cardio,1831,21,9.6122,0.5143,0.5143,0.5429,0.4857,0.4857,0.4143,...,0.5143,0.5857,0.5143,0.4857,0.5857,0.4571,0.5,0.5,0.5,0.5286
0,glass,214,9,4.2056,0.2,0.2,0.2,0.2,0.2,0.2,...,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2
0,ionosphere,351,33,35.8974,0.5435,0.5435,0.5435,0.5435,0.5435,0.5652,...,0.587,0.5435,0.587,0.5652,0.5435,0.5652,0.5217,0.5435,0.5652,0.5217
0,letter,1600,32,6.25,0.0732,0.0732,0.0732,0.0732,0.0488,0.0976,...,0.0488,0.0976,0.0976,0.0976,0.0976,0.0732,0.0732,0.0488,0.0488,0.0732
0,lympho,148,18,4.0541,0.6667,1.0,1.0,1.0,0.6667,1.0,...,0.6667,1.0,1.0,1.0,0.6667,1.0,1.0,1.0,1.0,1.0
0,mnist,7603,100,9.2069,0.2889,0.3,0.237,0.3185,0.2852,0.3111,...,0.3519,0.3333,0.2519,0.3556,0.3,0.2778,0.2296,0.2222,0.2889,0.2926
0,musk,3062,166,3.1679,0.9024,0.9512,0.7805,0.9268,1.0,0.8049,...,0.9268,1.0,0.9268,1.0,1.0,0.9756,1.0,0.9756,0.9024,1.0
0,optdigits,5216,64,2.8758,0.0462,0.0769,0.0462,0.0154,0.0308,0.0462,...,0.0308,0.0,0.0308,0.0462,0.0308,0.0308,0.0154,0.0154,0.0308,0.0769
0,pendigits,6870,16,2.2707,0.1935,0.2742,0.2419,0.2419,0.1774,0.2742,...,0.2581,0.2419,0.371,0.2742,0.2258,0.2258,0.2581,0.2581,0.2581,0.2581
